In [39]:
def equalizer(M_alpha, M_beta): 
    M = M_alpha + (-1)*M_beta
    M_null = M.nullspace()
    
    Ker_basis = []
    for i in range(len(M_null)):
        basis_vec = []
        for x in M_null[i]:
            basis_vec.append(x)
        Ker_basis.append(basis_vec)

    
    C1_basis = []
    for i in range(len(M.row(0))):
        basis_vec = []
        for j in range(len(M.row(0))):
            if(j == i):
                basis_vec.append(1)
            else:
                basis_vec.append(0)
        C1_basis.append(basis_vec)
        
        
   # print("Ker_basis", *Ker_basis)
   # print("C_1 basis", *C1_basis)
    
    Poly_C = Polyhedron(rays = C1_basis, base_ring = RDF)
    Poly_K = Polyhedron(lines = Ker_basis, base_ring = RDF)
    Eq = Poly_C.intersection(Poly_K)
    
    return Eq


In [40]:
def homology(G):
    V_E = G.edges() #V_E: [(1, 2, 'a'), (1, 3, 'b'), (3, 2, 'c')]
    vertices = []
    edges = []
    delta_positive = {}
    delta_negative = {}

    for tuple in V_E:
        for i in range(3):
            if i != 2:
                if tuple[i] not in vertices:
                    vertices.append(tuple[i])
            else:
                edges.append(tuple[i])

        delta_positive[tuple[2]] = tuple[1]
        delta_negative[tuple[2]] = tuple[0]

#     print(vertices)
#     print(edges)
#     print(delta_positive)
#     print(delta_negative)

    def get_M_Representation(vw_dic, w_list):
      M_col = []
      for v in vw_dic:
        linear_comb = []
        for w in w_list:
            if w == vw_dic[v]:
                linear_comb.append(1)
            else:
                linear_comb.append(0)

        M_col.append(linear_comb)

      return M_col

    M_positive_col = []
    M_negative_col = []
    M_positive_col = get_M_Representation(delta_positive, vertices)
    M_negative_col = get_M_Representation(delta_negative, vertices)

    M_positive = Matrix(M_positive_col).T
    M_negative = Matrix(M_negative_col).T

    graph_equalizer = equalizer(M_positive, M_negative) #how to keep it seperate from this homology function?

    return graph_equalizer.rays_list()





In [41]:
#Example: getting the equalizer of two mappings using their matrix representation
from sympy import *
M1 = Matrix([[1, 1, 0, 0], [0, 0, 1, 1]])
M2 = Matrix([[0, 0, 1, 1], [1, 1, 0, 0]])
equalizer_example = equalizer(M1, M2)
print(equalizer_example)
print(equalizer_example.rays_list())


A 3-dimensional polyhedron in RDF^4 defined as the convex hull of 1 vertex and 4 rays
[[1.0, 0.0, 0.0, 1.0], [1.0, 0.0, 1.0, 0.0], [0.0, 1.0, 1.0, 0.0], [0.0, 1.0, 0.0, 1.0]]


In [42]:
#Example calculating H1 of a directed graph. We input the directed graph object by specifying all the edges.
G_example = DiGraph({'A': {'B': ['a', 'h']}, 'B': {'E': ['c']}, 'C': {'B' : ['g']}, 'D' : {'A':['d'], 'E' : ['e']}, 'E' : {'B' : ['b'], 'C' : ['f']}})
G_homology = homology(G_example)
print(G_homology)
#result: get the equalizer generated by [(c + b), (c + g + f)]

[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0]]
